# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
datafile = "output_data/cities.csv"
cities_df = pd.read_csv(datafile)
cities_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Cape Town,0,ZA,1589156682,93,-33.93,18.42,288.71,4.10
1,1,Yulara,93,AU,1589156806,35,-25.24,130.99,287.15,7.70
2,2,Mumford,100,GH,1589156806,77,5.26,-0.76,299.37,4.68
3,3,Kutum,0,SD,1589156807,23,14.20,24.67,292.92,3.13
4,4,Khatanga,93,RU,1589156807,98,71.97,102.50,271.56,2.73


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
locations = cities_df[["Lat","Lng"]]
Humid_float = cities_df["Humidity"]

map_disp = gmaps.figure()


heat_layer = gmaps.heatmap_layer(locations, weights=Humid_float,
                           dissipating=False, max_intensity=100, point_radius=2)

map_disp.add_layer(heat_layer)

map_disp

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
c_df = cities_df.loc[(cities_df['Max Temp']>250) & (cities_df['Max Temp']<280) & 
                     (cities_df['Humidity']<60) & (cities_df['Cloudiness']<90)]
c_df = c_df.dropna()
c_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
136,136,Yellowknife,20,CA,1589156825,44,62.46,-114.35,277.15,4.10
291,291,Halifax,65,CA,1589156845,49,44.65,-63.57,278.71,1.34
385,385,Flin Flon,20,CA,1589156858,25,54.77,-101.86,276.15,3.60
436,436,Husavik,8,IS,1589156865,45,66.04,-17.34,279.15,13.90
528,528,Alberton,89,CA,1589156876,35,46.82,-64.07,277.59,9.12


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df = c_df
hotel_list = []
hotels_by_city = []
# hotel_df.head()

citylat = hotel_df['Lat'].to_list()
citylng = hotel_df['Lng'].to_list()

# hotel_df['Lat']
# hotel_df['Lng']

# citylng


url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?location="
radius = "&radius=5000"
placetype = "&type=hotel&keyword=hotel"
key = "&key=" + g_key

for x in range (len(citylng)):
    hotel_data = requests.get(url + str(citylat[x]) + "," + str(citylng[x]) + radius + placetype + key)
    hotel_list.append(hotel_data.json())
    try:
        hotels_by_city.append((hotel_data.json())['results'][0]['name'])
    except:
        hotels_by_city.append("No Hotel Available")

In [6]:
hotels_by_city

['The Explorer Hotel',
 'Halifax Marriott Harbourfront Hotel',
 'Royal Hotel',
 'Húsavík Hotels',
 'Briarwood Coastal Cottages & Lodge']

In [7]:
hotel_df["Hotel"] = hotels_by_city

hotel_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel
136,136,Yellowknife,20,CA,1589156825,44,62.46,-114.35,277.15,4.10,The Explorer Hotel
291,291,Halifax,65,CA,1589156845,49,44.65,-63.57,278.71,1.34,Halifax Marriott Harbourfront Hotel
385,385,Flin Flon,20,CA,1589156858,25,54.77,-101.86,276.15,3.60,Royal Hotel
436,436,Husavik,8,IS,1589156865,45,66.04,-17.34,279.15,13.90,Húsavík Hotels
528,528,Alberton,89,CA,1589156876,35,46.82,-64.07,277.59,9.12,Briarwood Coastal Cottages & Lodge


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
# hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
hotel_locations = hotel_df[["Lat", "Lng"]]

In [8]:
# Add marker layer ontop of heat map
hotel_locations = hotel_df[["Lat", "Lng"]]

hotel_layer = gmaps.marker_layer(hotel_locations)

map_disp.add_layer(hotel_layer)

# Display Map

map_disp

Figure(layout=FigureLayout(height='420px'))